In [46]:
import pandas as pd
from haversine import haversine

In [33]:
df = pd.read_csv('단속장소_최종중복제거.csv', encoding= 'cp949', index_col = 0)

In [35]:
df

,단속장소,X,Y,단속건수
0,서울특별시 양천구 목동 901-3,126.872374,37.537457,48
1,서울특별시 양천구 신정동 212-1,126.866868,37.509881,117
2,서울특별시 양천구 신정7 목동파크자이아파트 정문,126.855695,37.515174,556
3,서울특별시 양천구 신정동 신정기독교 백화점,126.852776,37.519932,261
4,서울특별시 양천구 신정동 311,126.857833,37.514988,131
...,...,...,...,...
135,서울특별시 양천구 신월동 서울 양천구 신월동 216,126.835559,37.526068,38
136,서울특별시 양천구 신정동 홍익병원,126.863634,37.528603,123
137,서울특별시 양천구 목동 현대파크빌,126.867165,37.542323,143
138,서울특별시 양천구 목동 목1동 서가앤쿡 목동점,126.869250,37.525320,237


### X좌표의 최소값과 최대값의 평균

In [38]:
x_cen = (df['X'].min() + df['X'].max())/2

###  Y좌표의 최소값과 최대값의 평균

In [39]:
y_cen = (df['Y'].min() + df['Y'].max())/2

### 각 좌표의 넘버링을 위해
### x,y 최대최소좌표의 중간값을 이용해 지역을 4분할

In [45]:
df1 = df[(df['X'] <= x_cen) & (df['Y'] >= y_cen) ] # 좌측 상단
df2 = df[(df['X'] <= x_cen) & (df['Y'] < y_cen) ] # 죄측 하단
df3 = df[(df['X'] > x_cen) & (df['Y'] < y_cen) ] # 우측 하단
df4 = df[(df['X'] > x_cen) & (df['Y'] >= y_cen)] # 우측 상단

In [52]:
for i in range(1,5):
    globals()[f'df{i}'].reset_index(inplace = True, drop = True)

### 4분할 별로 거리찍는 기준점

#### 좌측 상단 : y값이 가장 큰 좌표
#### 좌측 하단 : x값이 가장 작은 좌표
#### 우측 하단 : x값이 가장 작은 좌표
#### 우측 상단 : y값이 가장 작은 좌표

In [193]:
p1 = df1[df1['Y'] == df1['Y'].max()]
p2 = df2[df2['X'] == df2['X'].min()]
p3 = df3[df3['X'] == df3['X'].min()]
p4 = df4[df4['Y'] == df4['Y'].min()]

In [194]:
haversine((p2['X'], p2['Y']), (df1['X'][0], df1['Y'][0])) * 1000

1463.968155557792

In [195]:
for i in range(1,5):
    globals()[f'df{i}']['dis_from_first'] = 0

### 좌표 기준점에서 4분할 된 구역의 각 좌표까지 거리

In [196]:
def set_dis_from_first(df, point):
    for i in range(len(df)):
        df['dis_from_first'][i] = haversine((point['X'], point['Y']), (df['X'][i], df['Y'][i])) * 1000

In [197]:
set_dis_from_first(df1, p1)
set_dis_from_first(df2, p2)
set_dis_from_first(df3, p3)
set_dis_from_first(df4, p4)

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/98491907.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dis_from_first'][i] = haversine((point['X'], point['Y']), (df['X'][i], df['Y'][i])) * 1000


### df1 ~ 4 'label'컬럼에 1 ~ 4 (x, y좌표 위치 기준으로 4분할 한 위치 표시)

In [198]:
for i in range(1,5):
    globals()[f'df{i}']['label'] = i

### dis_from_first 컬럼으로 sort

In [199]:
for i in range(1,5):
    globals()[f'df{i}'] = globals()[f'df{i}'].sort_values('dis_from_first')

In [200]:
for i in range(1,5):
    globals()[f'df{i}'].reset_index(inplace = True, drop = True)

### df1 ~ df4 apppend

In [201]:
df_final = df1.append(df2)
df_final = df_final.append(df3)
df_final = df_final.append(df4)

In [202]:
df_final.reset_index(inplace = True, drop= True)

In [203]:
df_final['number'] = 0
for i in range(len(df_final)):
    df_final['number'][i] = df_final.index[i]

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/460159449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['number'][i] = df_final.index[i]


In [204]:
df_final

,단속장소,X,Y,단속건수,dis_from_first,label,number
0,서울특별시 양천구 신월3동신원초교,126.829355,37.545372,83,0,1,0
1,서울특별시 양천구 신월동 수명산롯데캐슬아파트,126.828413,37.544168,44,132,1,1
2,서울특별시 양천구 신월동 신월중,126.828963,37.541024,91,293,1,2
3,서울특별시 양천구 신월동 강서경찰서,126.830222,37.539764,98,386,1,3
4,서울특별시 양천구 신월동 273,126.828015,37.537837,46,523,1,4
...,...,...,...,...,...,...,...
135,서울특별시 양천구 목동 901,126.879447,37.539389,76,2128,4,135
136,서울특별시 양천구 목목동5단지아파트,126.880367,37.534470,100,2134,4,136
137,서울특별시 양천구 목5동 현대파크빌,126.880252,37.535853,159,2142,4,137
138,서울특별시 양천구 목동 912,126.881859,37.535143,63,2306,4,138


In [205]:
def hav_dis(df, a, b):
    h = (df['X'][a], df['Y'][a])
    j = (df['X'][b], df['Y'][b])
    dis = haversine(h,j) * 1000
    return dis

In [206]:
def set_dis_to_next_num(df):
    for i in range(len(df)-1):
        d = hav_dis(df, i, i+1)
        df['dis'][i] = d
    return df

In [207]:
df_final['dis'] = 0

In [208]:
set_dis_to_next_num(df_final)

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/3761305484.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dis'][i] = d


,단속장소,X,Y,단속건수,dis_from_first,label,number,dis
0,서울특별시 양천구 신월3동신원초교,126.829355,37.545372,83,0,1,0,132
1,서울특별시 양천구 신월동 수명산롯데캐슬아파트,126.828413,37.544168,44,132,1,1,218
2,서울특별시 양천구 신월동 신월중,126.828963,37.541024,91,293,1,2,163
3,서울특별시 양천구 신월동 강서경찰서,126.830222,37.539764,98,386,1,3,277
4,서울특별시 양천구 신월동 273,126.828015,37.537837,46,523,1,4,74
...,...,...,...,...,...,...,...,...
135,서울특별시 양천구 목동 901,126.879447,37.539389,76,2128,4,135,343
136,서울특별시 양천구 목목동5단지아파트,126.880367,37.534470,100,2134,4,136,93
137,서울특별시 양천구 목5동 현대파크빌,126.880252,37.535853,159,2142,4,137,184
138,서울특별시 양천구 목동 912,126.881859,37.535143,63,2306,4,138,260


df_final.to_csv('4분할.csv', encoding = 'cp949')

### numbering 기준 df 3분할

In [209]:
len_div_by_3 = df_final['number'].max() // 3

In [210]:
df_final['label_by_3'] = 0

In [211]:
df_final.loc[ :len_div_by_3]['label_by_3'] = 1

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/3939479114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[ :len_div_by_3]['label_by_3'] = 1


In [212]:
df_final.loc[len_div_by_3 + 1 : len_div_by_3 * 2]['label_by_3'] = 2

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/781483705.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[len_div_by_3 + 1 : len_div_by_3 * 2]['label_by_3'] = 2


In [213]:
df_final.loc[len_div_by_3*2 + 1 : ]['label_by_3'] = 3

C:\Users\bany9\AppData\Local\Temp/ipykernel_2476/4216261111.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.loc[len_div_by_3*2 + 1 : ]['label_by_3'] = 3


In [214]:
df_final.to_csv('최종3분할.csv', encoding = 'cp949')

In [218]:
df_final.dis.sum()

58058